In [2]:
!pip install openfermion
!pip install openfermionpyscf

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.2 MB/s eta 0:00:00
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=4c0a5af66fe74f6d6cf47cc9ad4cae4be321d35c6b5686fe2e8abdb80bbe0bf3
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 22.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import openfermion as of
from scipy import linalg
from scipy.linalg import expm
import matplotlib.pyplot as plt
from openfermion.chem import MolecularData

from openfermion.transforms import (
    get_fermion_operator,
    jordan_wigner)
from openfermionpyscf import run_pyscf
from openfermion.linalg import LinearQubitOperator

from tqdm.notebook import tqdm

## H4 molecule

In [4]:
multiplicity = 1
basis = "sto-3g"

h1, h2, h3, h4 = 0, 0.735, 1.535, 2.135


geometry = [('H', (0., 0., h1)), ('H', (0., 0., h2)),('H', (0., 0., h3)),('H', (0., 0., h4))]


description = "h4"
molecule = MolecularData(geometry, basis, multiplicity, description=description)
molecule = run_pyscf(molecule, run_mp2=True, run_cisd=True, run_ccsd=True, run_fci=True)

In [5]:
molHam = molecule.get_molecular_hamiltonian()

fermionHam = get_fermion_operator(molHam)

H = jordan_wigner(fermionHam)


print('Number of qubits:', LinearQubitOperator(H).n_qubits)

Number of qubits: 8


In [6]:
# some helper functions

def channel(H, t, rho):
    #return expm(1.j*H*t).dot(rho).dot(expm(-1.j*H*t))
    return expm(1.j*H*t) @ rho @ expm(-1.j*H*t)

def trace_norm(A):
    return .5*sum(linalg.svdvals(A)) # same as np.linalg.norm(A, ord=2)

# qDRIFT Algorithm

In [7]:
# qDRIFT algorithm

def qDRIFT(n, H, t, eps, rho0):

    L = len(H.terms)

    h = [list(H.terms.values())[j] for j in range(L)]

    lam = sum(np.abs(h))

    H_list = [list(H)[j]/abs(h[j]) for j in range(L)]

    N = int(2 * lam**2 * t**2 / eps)

    print('lambda:', lam, 'N:', N)

    rho = rho0

    tau = lam * t / N

    for j in tqdm(range(N)):

        local_H = np.random.choice(H_list, p = np.abs(h)/lam)

        local_H_mtx = of.get_sparse_operator(local_H,n).toarray()

        rho = channel(local_H_mtx, tau, rho)

    return rho

In [8]:
# Let's check qDRIFT for simple case. Inital state is the completely mixed state.

n = 8
rho0 = (1/2**n)*np.ones((2**n,2**n))
eps = 0.01
t=1

In [9]:
rho_qdrift = qDRIFT(n, H, t, eps, rho0)

lambda: 10.614937998461802 N: 22535


  0%|          | 0/22535 [00:00<?, ?it/s]

In [15]:
H_mtx = of.get_sparse_operator(H,n).toarray()

In [12]:
rho_exact = channel(H_mtx, t, rho0)

rho_qdrift_error = rho_exact - rho_qdrift

qdrift_error = trace_norm(rho_qdrift_error)

print('qDRIFT error:', qdrift_error)

qDRIFT error: 0.055244423676344176


# Superlinear qDRIFT Algorithm (Old)

In [9]:
def shortTimeLinearQDrift(n, H, t, eps, rho0, num_steps):

    L = len(H.terms)

    h = [list(H.terms.values())[j] for j in range(L)]

    lam = sum(np.abs(h))

    H_list = [list(H)[j]/abs(h[j]) for j in range(L)]

    N = int(2 * lam**2 * t**2 / eps)

    print('lambda:', lam, 'N:', N)

    tau = lam * t / N

    p = np.abs(h)/lam

    #qDRIFT evolution for num_steps * tau time
    print('short-time qDRIFT evolution')

    rho_qDRIFT = rho0
    for s in tqdm(range(num_steps)):
        rho_qDRIFT = sum(p[i]*channel(of.get_sparse_operator(H_list[i],n).toarray(), tau, rho_qDRIFT) for i in range(L))

    # superLinear qDRIFT evolution for num_steps * tau time
    print('short-time superlinear qDRIFT evolution')

    rho = rho0
    for s in tqdm(range(num_steps)):

        rho_right = sum(p[i]*channel(of.get_sparse_operator(H_list[i],n).toarray(), tau, rho) for i in range(L))

        rho = sum(p[j]*channel(of.get_sparse_operator(H_list[j],n).toarray(), tau/2, rho) for j in range(L))

        rho = sum(p[i]*channel(of.get_sparse_operator(H_list[i],n).toarray(), tau/2, rho) for i in range(L))

        rho = 2*rho - rho_right

    rho_superlinear_qDRIFT = rho

    return rho_qDRIFT, rho_superlinear_qDRIFT

## Comapring errors for one step

In [14]:
rho_qDRIFT_oneStep, rho_superlinear_qDRIFT_oneStep = shortTimeLinearQDrift(n, H, t, eps, rho0, 1)

lambda: 10.614937998461802 N: 22535
short-time qDRIFT evolution


  0%|          | 0/1 [00:00<?, ?it/s]

short-time superlinear qDRIFT evolution


  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
L = len(H.terms)

h = [list(H.terms.values())[j] for j in range(L)]

lam = sum(np.abs(h))

tau = t/int(2 * lam**2 * t**2 / eps)

rho_exact_oneStep = channel(H_mtx, tau, rho0)

qDrift_error_oneStep = trace_norm(rho_exact_oneStep - rho_qDRIFT_oneStep)

superlinear_qDrift_error_oneStep = trace_norm(rho_exact_oneStep - rho_superlinear_qDRIFT_oneStep)

print('One step error for qDRIFT', qDrift_error_oneStep)
print('One step error for superlinear qDRIFT', superlinear_qDrift_error_oneStep)

## Comparing errors for 10 steps

In [10]:
rho_qDRIFT_tenStep, rho_superlinear_qDRIFT_tenStep = shortTimeLinearQDrift(n, H, t, eps, rho0, 10)

lambda: 10.614937998461809 N: 22535
short-time qDRIFT evolution


  0%|          | 0/10 [00:00<?, ?it/s]

short-time superlinear qDRIFT evolution


  0%|          | 0/10 [00:00<?, ?it/s]

In [16]:
rho_exact_tenStep = channel(H_mtx, 10*tau, rho0)

qDrift_error_tenStep = trace_norm(rho_exact_tenStep - rho_qDRIFT_tenStep)

superlinear_qDrift_error_tenStep = trace_norm(rho_exact_tenStep - rho_superlinear_qDRIFT_tenStep)

print('One step error for qDRIFT', qDrift_error_tenStep)
print('One step error for superlinear qDRIFT', superlinear_qDrift_error_tenStep)

One step error for qDRIFT 1.9970831929321516e-06
One step error for superlinear qDRIFT 5.1194909105620653e-11


## Comparins errors for 50 steps

In [ ]:
rho_qDRIFT_50Step, rho_superlinear_qDRIFT_50Step = shortTimeLinearQDrift(n, H, t, eps, rho0, 50)

In [ ]:
rho_exact_50Step = channel(H_mtx, 50*tau, rho0)

qDrift_error_50Step = trace_norm(rho_exact_50Step - rho_qDRIFT_50Step)

superlinear_qDrift_error_50Step = trace_norm(rho_exact_50Step - rho_superlinear_qDRIFT_50Step)

print('One step error for qDRIFT', qDrift_error_50Step)
print('One step error for superlinear qDRIFT', superlinear_qDrift_error_50Step)

# 2nd Order Linear qDRIFT

In [22]:
def commutator(A, B):
    return A @ B - B @ A

def secondOrderLinearQDrift(n, H, t, eps, rho0, num_steps):
    """
    Second-order channel method for approximating time evolution over t.

    Parameters:
    - n: Num qubits.
    - H: Hamiltonian (QubitOperator).
    - t: Total time for the evolution.
    - eps: Desired error (epsilon)
    - rho0: Initial density matrix.
    - num_steps: Number of time steps in the evolution.

    Returns:
    - rho: Density matrix after applying the second-order channel over t.
    """
    # Usual setup
    L = len(H.terms)
    h = [list(H.terms.values())[j] for j in range(L)]
    lam = sum(np.abs(h))
    H_list = [list(H)[j]/abs(h[j]) for j in range(L)]

    N = int(2 * lam**2 * t**2 / eps)
    print('lambda:', lam, 'N:', N)
    tau = lam * t / N
    p = np.abs(h)/lam

    # qDRIFT evolution for num_steps * tau time
    print('short-time qDRIFT evolution')

    rho_qDRIFT = rho0
    for s in tqdm(range(num_steps)):
        rho_qDRIFT = sum(p[i]*channel(of.get_sparse_operator(H_list[i],n).toarray(), tau, rho_qDRIFT) for i in range(L))

    # Second order Linear qDRIFT evolution for num_steps * tau time
    print('short-time 2nd order Linear qDRIFT evolution')

    rho = rho0
    # Loop over num_steps for the evolution
    for _ in tqdm(range(num_steps), desc="Applying second-order channel"):

        # 2nd order correction
        corr = sum(
            p[j] * of.get_sparse_operator(H_list[j], n).toarray() @ rho @
            of.get_sparse_operator(H_list[j], n).toarray()
            for j in range(L)
        )

        # First part: the probabilistic mix of Hamiltonian channels (rho_E1)
        H_total = of.get_sparse_operator(H, n).toarray()

        rho_E1 = (1 - 2*tau**2) * rho + 2*1j*(tau/lam)*(commutator(H_total, rho)) \
        - (tau**2 / lam**2) * commutator(H_total, commutator(H_total, rho)) + 2*tau**2 * \
        corr

        # Second part: the second-order correction (rho_E2)
        rho_E2 = corr

        # Combine them according to the corrected formula
        rho = 0.5 * (1 + 2 * tau**2) * rho + 0.5 * rho_E1 - tau**2 * rho_E2

    rho_secondorder_linear_qDRIFT = rho

    return rho_qDRIFT, rho_secondorder_linear_qDRIFT

## Comparing errors for one step

In [23]:
rho_qDRIFT_oneStep, rho_secondorder_linear_qDRIFT_oneStep = secondOrderLinearQDrift(n, H, t, eps, rho0, 1)

lambda: 10.614937998461802 N: 22535
short-time qDRIFT evolution


  0%|          | 0/1 [00:00<?, ?it/s]

short-time 2nd order Linear qDRIFT evolution


Applying second-order channel:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
L = len(H.terms)

h = [list(H.terms.values())[j] for j in range(L)]

lam = sum(np.abs(h))

tau = t/int(2 * lam**2 * t**2 / eps)

rho_exact_oneStep = channel(H_mtx, tau, rho0)

qDrift_error_oneStep = trace_norm(rho_exact_oneStep - rho_qDRIFT_oneStep)

secondorder_linear_qDrift_error_oneStep = trace_norm(rho_exact_oneStep - rho_secondorder_linear_qDRIFT_oneStep)

print('One step error for qDRIFT', qDrift_error_oneStep)
print('One step error for 2nd order linear qDRIFT', secondorder_linear_qDrift_error_oneStep)

One step error for qDRIFT 1.9970850323719459e-07
One step error for 2nd order linear qDRIFT 2.992524422228183e-13


## Comparing errors for 10 steps

In [26]:
rho_qDRIFT_tenStep, rho_secondorder_linear_qDRIFT_tenStep = secondOrderLinearQDrift(n, H, t, eps, rho0, 10)

lambda: 10.614937998461802 N: 22535
short-time qDRIFT evolution


  0%|          | 0/10 [00:00<?, ?it/s]

short-time 2nd order Linear qDRIFT evolution


Applying second-order channel:   0%|          | 0/10 [00:00<?, ?it/s]

In [27]:
L = len(H.terms)

h = [list(H.terms.values())[j] for j in range(L)]

lam = sum(np.abs(h))

tau = t/int(2 * lam**2 * t**2 / eps)

rho_exact_tenStep = channel(H_mtx, 10*tau, rho0)

qDrift_error_tenStep = trace_norm(rho_exact_tenStep - rho_qDRIFT_tenStep)

secondorder_linear_qDrift_error_tenStep = trace_norm(rho_exact_tenStep - rho_secondorder_linear_qDRIFT_tenStep)

print('10 step error for qDRIFT', qDrift_error_tenStep)
print('10 step error for 2nd order linear qDRIFT', secondorder_linear_qDrift_error_tenStep)

10 step error for qDRIFT 1.9970831943121686e-06
10 step error for 2nd order linear qDRIFT 2.9862571175659855e-12
